# 결측치 분석만 하는 코드
- 결측치 개수/비율 확인
- 컬럼별/날짜별/타입별 결측치 패턴 분석
- 결측치 있는 암호화폐 찾기
- 결측치 처리/제거는 안함

# 결과
### 초반기간엔 코인특성상 데이터가없는경우가많아서 결측치가 많음
### 나머지는 채워졌으므로 따로 전처리는 하지않아도됨

# Data Load

In [1]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import warnings
import utils as U
warnings.filterwarnings('ignore')

In [2]:
# 저장된 데이터 불러오기
crypto_data, excluded_info, summary_info = U.load_cleaned_crypto_data("/workspace/AI모델/projects/coin/data/final/with_strategies")

# 불러온 데이터 확인
print(f"로딩된 종목 수: {len(crypto_data)}")
for symbol, df in list(crypto_data.items())[:3]:
    print(f"{symbol}: {len(df)}행, {df.index[0].date()} ~ {df.index[-1].date()}")

# 상세 정보 출력
U.get_data_info(crypto_data, detailed=True)

# ML 데이터셋 로드
ml_dataset = U.CryptoDataLoader.load_ml_dataset("/workspace/AI모델/projects/coin/data/final")

📂 데이터 로딩 시작: /workspace/AI모델/projects/coin/data/final/with_strategies
📊 발견된 파일 수: 100
✅ 성공적으로 로드된 심볼: 100개
❌ 제외된 심볼: 0개
🔥 로딩 에러: 0개
로딩된 종목 수: 100
1INCH: 1725행, 2020-12-25 ~ 2025-09-14
AAVE: 1809행, 2020-10-02 ~ 2025-09-14
ADA: 2867행, 2017-11-09 ~ 2025-09-14

📊 데이터 정보 요약
총 심볼 수: 100
컬럼 수: 178
전략 시그널 수: 41
데이터 기간: 2015-01-01 ~ 2025-09-14
평균 데이터 포인트: 2258일

📋 심볼별 상세 정보:
  1INCH: 1725행, 2020-12-25 ~ 2025-09-14
  AAVE: 1809행, 2020-10-02 ~ 2025-09-14
  ADA: 2867행, 2017-11-09 ~ 2025-09-14
  ALGO: 2278행, 2019-06-21 ~ 2025-09-14
  ANKR: 2385행, 2019-03-06 ~ 2025-09-14
  API3: 1748행, 2020-12-02 ~ 2025-09-14
  APT21794: 1062행, 2022-10-19 ~ 2025-09-14
  ARB11841: 907행, 2023-03-23 ~ 2025-09-14
  ASTR: 1336행, 2022-01-18 ~ 2025-09-14
  ATOM: 2377행, 2019-03-14 ~ 2025-09-14
  ... 및 90개 더

🎯 전략 시그널 목록:
  이동평균: 4개
  MACD: 6개
  RSI: 8개
  오실레이터: 6개
  가격패턴: 3개
  거래량: 4개
  모멘텀: 2개
  복합: 2개
✅ ML 데이터셋 로드 완료: 220052행, 47개 특성
📊 타겟 분포 - SELL(0): 70207, HOLD(1): 82407, BUY(2): 67438


In [3]:
display(ml_dataset)

,Open,High,Low,Close,Volume,MA_7,MA_20,MA_50,EMA_12,EMA_26,...,Momentum_Signal,Volatility_Signal,Final_Composite_Signal,Buy_Signal_Count,Sell_Signal_Count,Net_Signal_Score,Symbol,Date,Future_Return_7d,Target_Label
0,5.635379,5.754587,5.143051,5.285911,340232335.0,5.212768,4.434244,2.619175,4.963742,4.166663,...,1,2,0,3,5,-2,1INCH,2021-02-12,-0.127671,0
1,5.276110,5.997387,5.166696,5.709677,408052842.0,5.278580,4.595910,2.686798,5.078524,4.283263,...,1,1,1,2,2,0,1INCH,2021-02-13,-0.093616,0
2,5.707148,6.243050,5.139567,5.287905,362249720.0,5.325314,4.742008,2.760618,5.110742,4.359066,...,1,1,0,1,6,-5,1INCH,2021-02-14,-0.042054,1
3,5.270541,5.498158,4.307358,4.949254,404435889.0,5.268800,4.855807,2.838361,5.085894,4.403537,...,1,2,1,2,3,-1,1INCH,2021-02-15,-0.081672,0
4,4.946544,5.161109,4.643222,4.749412,240797372.0,5.218169,4.967290,2.911148,5.034122,4.429565,...,1,2,1,3,2,1,1INCH,2021-02-16,-0.233317,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220047,0.261622,0.283561,0.257863,0.279276,52511097.0,0.259826,0.253016,0.258609,0.258220,0.255308,...,2,1,2,4,0,4,ZRX,2025-09-03,0.000433,1
220048,0.279276,0.279778,0.260498,0.262081,36688030.0,0.259421,0.253948,0.258413,0.258814,0.255810,...,1,2,0,4,6,-2,ZRX,2025-09-04,0.062996,2
220049,0.262081,0.277024,0.261904,0.270450,46257847.0,0.262923,0.254683,0.258305,0.260604,0.256894,...,1,1,1,2,1,1,ZRX,2025-09-05,0.023775,1
220050,0.270450,0.278908,0.267564,0.268764,35571944.0,0.265479,0.255407,0.258110,0.261860,0.257773,...,1,1,0,1,5,-4,ZRX,2025-09-06,0.058576,2


In [4]:
print(crypto_data["BTC"].info())
print(crypto_data)



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3910 entries, 2015-01-01 to 2025-09-14
Columns: 178 entries, Open to Final_Composite_Signal
dtypes: float64(138), int64(40)
memory usage: 5.3 MB
None
{'1INCH':                 Open      High       Low     Close       Volume      MA_2  \
Date                                                                        
2020-12-25  1.602781  2.918094  1.375262  2.328544  638225549.0       NaN   
2020-12-26  2.318946  2.434916  1.582264  1.596896  237653073.0  1.962720   
2020-12-27  1.597744  1.630781  1.056657  1.062112  183978307.0  1.329504   
2020-12-28  1.064454  1.254447  1.049051  1.110076  119337529.0  1.086094   
2020-12-29  1.110460  1.111282  0.761151  0.887798  137916899.0  0.998937   
...              ...       ...       ...       ...          ...       ...   
2025-09-10  0.254335  0.262860  0.253173  0.259931   20683287.0  0.257133   
2025-09-11  0.259931  0.267249  0.259562  0.266820   22066023.0  0.263376   
2025-09-12  0.266

# 종목별 결측치 2차 필터링

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def analyze_missing_data(crypto_data, detailed=True, top_n=10):
    """
    암호화폐별 결측치 상세 분석
    
    Parameters:
    -----------
    crypto_data : dict
        {심볼: DataFrame} 형태의 암호화폐 데이터
    detailed : bool
        상세 분석 여부 (컬럼별 결측치 포함)
    top_n : int
        상위 N개 심볼만 표시
    
    Returns:
    --------
    missing_summary : pd.DataFrame
        심볼별 결측치 요약
    detailed_results : dict
        상세 결측치 정보
    """
    
    print("=" * 60)
    print("🪙 암호화폐별 결측치 분석")
    print("=" * 60)
    
    # 전체 분석 결과 저장
    analysis_results = []
    detailed_results = {}
    
    for symbol in tqdm(crypto_data.keys(), desc="결측치 분석"):
        df = crypto_data[symbol]
        
        # 기본 통계
        total_cells = len(df) * len(df.columns)
        total_missing = df.isnull().sum().sum()
        missing_percentage = (total_missing / total_cells) * 100
        
        # 행별 결측치
        rows_with_missing = df.isnull().any(axis=1).sum()
        rows_missing_percentage = (rows_with_missing / len(df)) * 100
        
        # 완전히 비어있는 행
        completely_empty_rows = df.isnull().all(axis=1).sum()
        
        # 전략 시그널 관련 결측치
        signal_columns = [col for col in df.columns if '_Signal' in col]
        signal_missing = df[signal_columns].isnull().sum().sum() if signal_columns else 0
        
        # 기술지표 관련 결측치
        indicator_columns = [col for col in df.columns if any(x in col for x in ['MA_', 'EMA_', 'RSI_', 'MACD', 'BB_', 'ATR', 'MFI', 'Williams', 'CCI', 'Stoch'])]
        indicator_missing = df[indicator_columns].isnull().sum().sum() if indicator_columns else 0
        
        # 결과 저장
        result = {
            'symbol': symbol,
            'total_rows': len(df),
            'total_columns': len(df.columns),
            'total_missing': total_missing,
            'missing_percentage': missing_percentage,
            'rows_with_missing': rows_with_missing,
            'rows_missing_percentage': rows_missing_percentage,
            'completely_empty_rows': completely_empty_rows,
            'signal_columns': len(signal_columns),
            'signal_missing': signal_missing,
            'indicator_columns': len(indicator_columns),
            'indicator_missing': indicator_missing,
            'data_period_days': len(df),
            'start_date': df.index.min() if not df.empty else None,
            'end_date': df.index.max() if not df.empty else None
        }
        
        analysis_results.append(result)
        
        # 상세 분석 (컬럼별)
        if detailed:
            column_missing = df.isnull().sum()
            column_missing_pct = (column_missing / len(df)) * 100
            
            # 결측치가 있는 컬럼만
            missing_columns = column_missing[column_missing > 0]
            
            if len(missing_columns) > 0:
                detailed_results[symbol] = {
                    'missing_by_column': missing_columns.to_dict(),
                    'missing_pct_by_column': column_missing_pct[column_missing_pct > 0].to_dict(),
                    'signal_columns': signal_columns,
                    'indicator_columns': indicator_columns
                }
    
    # DataFrame으로 변환
    summary_df = pd.DataFrame(analysis_results)
    summary_df = summary_df.sort_values('missing_percentage', ascending=False)
    
    # 전체 요약 출력
    print(f"전체 암호화폐 수: {len(crypto_data)}")
    print(f"결측치가 있는 암호화폐: {(summary_df['total_missing'] > 0).sum()}개")
    print(f"완전히 깨끗한 암호화폐: {(summary_df['total_missing'] == 0).sum()}개")
    
    # 전체 통계
    total_missing_all = summary_df['total_missing'].sum()
    total_cells_all = (summary_df['total_rows'] * summary_df['total_columns']).sum()
    overall_missing_pct = (total_missing_all / total_cells_all) * 100
    
    print(f"전체 결측치 비율: {overall_missing_pct:.2f}%")
    
    # 전략/지표별 통계
    total_signals = summary_df['signal_missing'].sum()
    total_indicators = summary_df['indicator_missing'].sum()
    print(f"전략 시그널 결측치: {total_signals}개")
    print(f"기술지표 결측치: {total_indicators}개")
    print()
    
    # 결측치가 많은 상위 종목들
    top_missing = summary_df[summary_df['total_missing'] > 0].head(top_n)
    
    if len(top_missing) > 0:
        print(f"결측치가 많은 상위 {min(top_n, len(top_missing))}개 암호화폐:")
        print("-" * 100)
        print(f"{'심볼':<10} {'기간(일)':<8} {'결측치':<8} {'비율(%)':<8} {'결측행':<8} {'빈행':<6} {'시그널결측':<10} {'지표결측':<10}")
        print("-" * 100)
        
        for _, row in top_missing.iterrows():
            print(f"{row['symbol']:<10} {row['data_period_days']:<8} {row['total_missing']:<8} "
                  f"{row['missing_percentage']:<8.2f} {row['rows_with_missing']:<8} {row['completely_empty_rows']:<6} "
                  f"{row['signal_missing']:<10} {row['indicator_missing']:<10}")
        
        # 상세 정보 출력
        if detailed:
            print(f"\n상위 {min(5, len(top_missing))}개 암호화폐 컬럼별 상세:")
            print("=" * 80)
            
            for _, row in top_missing.head(5).iterrows():
                symbol = row['symbol']
                print(f"\n[{symbol}] - 총 {row['total_missing']}개 결측치 ({row['missing_percentage']:.2f}%)")
                print(f"  📊 데이터 기간: {row['start_date'].date()} ~ {row['end_date'].date()} ({row['data_period_days']}일)")
                print(f"  🎯 전략 시그널: {row['signal_columns']}개 (결측: {row['signal_missing']})")
                print(f"  📈 기술지표: {row['indicator_columns']}개 (결측: {row['indicator_missing']})")
                
                if symbol in detailed_results:
                    missing_cols = detailed_results[symbol]['missing_pct_by_column']
                    sorted_cols = sorted(missing_cols.items(), key=lambda x: x[1], reverse=True)
                    
                    print(f"  결측치 상위 컬럼:")
                    for col, pct in sorted_cols[:10]:  # 상위 10개 컬럼만
                        count = detailed_results[symbol]['missing_by_column'][col]
                        col_type = "📈지표" if col in detailed_results[symbol]['indicator_columns'] else \
                                  "🎯시그널" if col in detailed_results[symbol]['signal_columns'] else "💰기본"
                        print(f"    {col_type} {col:<25}: {count:>6}개 ({pct:>6.2f}%)")
                    
                    if len(sorted_cols) > 10:
                        print(f"    ... 외 {len(sorted_cols) - 10}개 컬럼")
    else:
        print("🎉 모든 암호화폐가 결측치 없이 깨끗합니다!")
    
    return summary_df, detailed_results

def get_missing_patterns(crypto_data, pattern_type='by_column'):
    """
    결측치 패턴 분석
    
    Parameters:
    -----------
    crypto_data : dict
        암호화폐 데이터
    pattern_type : str
        'by_column': 컬럼별 패턴, 'by_date': 날짜별 패턴, 'by_type': 타입별 패턴
    """
    
    print(f"\n📊 결측치 패턴 분석 ({pattern_type})")
    print("=" * 60)
    
    if pattern_type == 'by_column':
        # 전체 컬럼별 결측치 통계
        all_columns = set()
        for df in crypto_data.values():
            all_columns.update(df.columns)
        
        column_stats = {}
        for col in all_columns:
            missing_counts = []
            total_counts = []
            
            for symbol, df in crypto_data.items():
                if col in df.columns:
                    missing_counts.append(df[col].isnull().sum())
                    total_counts.append(len(df))
            
            if missing_counts:
                total_missing = sum(missing_counts)
                total_rows = sum(total_counts)
                column_stats[col] = {
                    'total_missing': total_missing,
                    'total_rows': total_rows,
                    'missing_percentage': (total_missing / total_rows) * 100,
                    'cryptos_affected': len([x for x in missing_counts if x > 0])
                }
        
        # 정렬 및 출력
        sorted_columns = sorted(column_stats.items(), 
                              key=lambda x: x[1]['missing_percentage'], 
                              reverse=True)
        
        print(f"{'컬럼명':<25} {'총결측치':<10} {'비율(%)':<10} {'영향코인':<10} {'타입':<10}")
        print("-" * 70)
        
        for col, stats in sorted_columns:
            if stats['missing_percentage'] > 0:
                # 컬럼 타입 분류
                if '_Signal' in col:
                    col_type = "🎯시그널"
                elif any(x in col for x in ['MA_', 'EMA_', 'RSI_', 'MACD', 'BB_', 'ATR', 'MFI', 'Williams', 'CCI', 'Stoch']):
                    col_type = "📈지표"
                elif col in ['Open', 'High', 'Low', 'Close', 'Volume']:
                    col_type = "💰기본"
                else:
                    col_type = "📊기타"
                
                print(f"{col:<25} {stats['total_missing']:<10} "
                      f"{stats['missing_percentage']:<10.2f} {stats['cryptos_affected']:<10} {col_type:<10}")
    
    elif pattern_type == 'by_date':
        # 날짜별 결측치 패턴 (최근 100일)
        all_dates = set()
        for df in crypto_data.values():
            all_dates.update(df.index)
        
        # 최근 100일만 샘플링
        recent_dates = sorted(all_dates)[-100:] if len(all_dates) > 100 else sorted(all_dates)
        
        date_missing = {}
        for date in recent_dates:
            total_missing = 0
            total_possible = 0
            
            for df in crypto_data.values():
                if date in df.index:
                    total_missing += df.loc[date].isnull().sum()
                    total_possible += len(df.columns)
            
            if total_possible > 0:
                date_missing[date] = (total_missing / total_possible) * 100
        
        # 결측치가 많은 날짜 상위 10개
        worst_dates = sorted(date_missing.items(), key=lambda x: x[1], reverse=True)[:10]
        
        if worst_dates:
            print(f"결측치가 많은 날짜 (최근 {len(recent_dates)}일 중):")
            print(f"{'날짜':<12} {'결측치비율(%)':<15}")
            print("-" * 30)
            
            for date, pct in worst_dates:
                if pct > 0:
                    print(f"{date.strftime('%Y-%m-%d'):<12} {pct:<15.2f}")
    
    elif pattern_type == 'by_type':
        # 컬럼 타입별 결측치 통계
        type_stats = {
            '💰 기본 가격/거래량': ['Open', 'High', 'Low', 'Close', 'Volume'],
            '📈 기술지표': [],
            '🎯 전략시그널': [],
            '📊 기타': []
        }
        
        # 모든 컬럼 분류
        all_columns = set()
        for df in crypto_data.values():
            all_columns.update(df.columns)
        
        for col in all_columns:
            if col in type_stats['💰 기본 가격/거래량']:
                continue
            elif '_Signal' in col:
                type_stats['🎯 전략시그널'].append(col)
            elif any(x in col for x in ['MA_', 'EMA_', 'RSI_', 'MACD', 'BB_', 'ATR', 'MFI', 'Williams', 'CCI', 'Stoch', 'Return', 'Volatility']):
                type_stats['📈 기술지표'].append(col)
            else:
                type_stats['📊 기타'].append(col)
        
        # 타입별 결측치 계산
        print(f"{'타입':<15} {'컬럼수':<8} {'총결측치':<10} {'평균비율(%)':<12}")
        print("-" * 50)
        
        for type_name, columns in type_stats.items():
            if not columns:
                continue
                
            total_missing = 0
            total_cells = 0
            
            for symbol, df in crypto_data.items():
                available_cols = [col for col in columns if col in df.columns]
                if available_cols:
                    type_missing = df[available_cols].isnull().sum().sum()
                    type_cells = len(df) * len(available_cols)
                    total_missing += type_missing
                    total_cells += type_cells
            
            avg_pct = (total_missing / total_cells * 100) if total_cells > 0 else 0
            print(f"{type_name:<15} {len(columns):<8} {total_missing:<10} {avg_pct:<12.2f}")

def quick_missing_check(crypto_data):
    """
    빠른 결측치 체크 (요약만)
    """
    
    print("⚡ 빠른 결측치 체크")
    print("=" * 40)
    
    clean_cryptos = []
    dirty_cryptos = []
    signal_issues = []
    
    for symbol, df in crypto_data.items():
        missing_count = df.isnull().sum().sum()
        
        # 전략 시그널 결측치 확인
        signal_columns = [col for col in df.columns if '_Signal' in col]
        signal_missing = df[signal_columns].isnull().sum().sum() if signal_columns else 0
        
        if missing_count == 0:
            clean_cryptos.append(symbol)
        else:
            missing_pct = (missing_count / (len(df) * len(df.columns))) * 100
            dirty_cryptos.append((symbol, missing_count, missing_pct))
            
            if signal_missing > 0:
                signal_issues.append((symbol, signal_missing))
    
    print(f"🟢 깨끗한 암호화폐: {len(clean_cryptos)}개")
    print(f"🟡 결측치 있는 암호화폐: {len(dirty_cryptos)}개")
    print(f"🔴 전략시그널 결측: {len(signal_issues)}개")
    
    if dirty_cryptos:
        # 가장 심한 5개
        worst_5 = sorted(dirty_cryptos, key=lambda x: x[2], reverse=True)[:5]
        print(f"\n가장 심한 5개 암호화폐:")
        for symbol, count, pct in worst_5:
            print(f"  {symbol}: {count}개 ({pct:.2f}%)")
    
    if signal_issues:
        print(f"\n전략시그널 결측치 문제:")
        for symbol, count in signal_issues[:5]:
            print(f"  {symbol}: {count}개 시그널 결측")


In [6]:
# 상세 분석
summary_df, detailed = analyze_missing_data(crypto_data, detailed=True, top_n=15)

# 패턴 분석
get_missing_patterns(crypto_data, 'by_column')  # 컬럼별
get_missing_patterns(crypto_data, 'by_type')    # 타입별
get_missing_patterns(crypto_data, 'by_date')    # 날짜별

# 빠른 체크
quick_missing_check(crypto_data)

🪙 암호화폐별 결측치 분석


결측치 분석: 100%|██████████| 100/100 [00:00<00:00, 234.88it/s]


전체 암호화폐 수: 100
결측치가 있는 암호화폐: 100개
완전히 깨끗한 암호화폐: 0개
전체 결측치 비율: 1.69%
전략 시그널 결측치: 3300개
기술지표 결측치: 116715개

결측치가 많은 상위 15개 암호화폐:
----------------------------------------------------------------------------------------------------
심볼         기간(일)    결측치      비율(%)    결측행      빈행     시그널결측      지표결측      
----------------------------------------------------------------------------------------------------
SHIB       1830     12247    3.76     1830     0      33         1380      
IOTA       716      3636     2.85     716      0      33         1165      
MNT27075   788      3780     2.69     788      0      33         1165      
SUI20947   866      3936     2.55     866      0      33         1165      
ARB11841   907      4018     2.49     907      0      33         1165      
APT21794   1062     4328     2.29     1062     0      33         1165      
OP         1281     4766     2.09     1281     0      33         1165      
ASTR       1336     4876     2.05     1336     0      33        